In [1]:
import numpy as np
import pandas as pd
# import plotly.express as px
# from autokeras import StructuredDataRegressor
from sklearn.model_selection import train_test_split
# from IPython.display import HTML
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

2022-08-31 20:37:16.194829: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import sys
import time
sys.path.insert(0, '/home/anish/ASC_ML')

In [21]:
from ASC_ML.networkbuilding import multiple_model_gen_v3 as multiple
from ASC_ML.networkbuilding import dataframe_extractor as de
from ASC_ML.networkbuilding import model_generation as model_gen
from ASC_ML.networkbuilding import model_optimization as model_opt
from ASC_ML.networkbuilding import model_stacking

In [4]:
df = pd.read_csv("/home/anish/Downloads/autokeras-regression-master/Data.csv")

In [5]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [6]:
# fig = px.scatter(df, x= "TV", y="sales")
# HTML(fig.to_html())

In [7]:
# fig = px.scatter(df, x= "radio", y="sales")
# HTML(fig.to_html())

In [8]:
# fig = px.scatter(df, x= "newspaper", y="sales")
# HTML(fig.to_html())

In [9]:
target_col = "sales"
X = df.loc[:, df.columns != target_col]
y = df.loc[:, target_col]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=2021)

In [11]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [12]:
y_train = y_train.reshape((y_train.shape[0],1))
y_test = y_test.reshape((y_test.shape[0],1))

In [13]:
start = time.time()
m = multiple.Multiple_Model_Gen_V3(X_train, y_train, X_test, y_test, 50, 64, input_shape = 3, 
                                   max_no_layers = 3, model_per_batch = 10, 
                                   save_dir = "/home/anish/ASC_ML_test_weights/")
m.get_model_confs()
m.get_best_models(save = False)
end = time.time()
print("Time Taken : ", (end-start))

2022-08-31 20:40:08.154893: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-08-31 20:40:08.155539: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-08-31 20:40:08.214974: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-31 20:40:08.215228: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.695GHz coreCount: 30 deviceMemorySize: 5.79GiB deviceMemoryBandwidth: 312.97GiB/s
2022-08-31 20:40:08.215272: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-08-31 20:40:08.217884: I tensorflow/stream_executor/platform/de


 
 

loss  :  10.594283103942871 , TEST :  11.34982967376709
output_layer_dense_32_32_loss  :  1.1226089000701904 , TEST :  1.2004287242889404
output_layer_dense_32_64_loss  :  1.1307462453842163 , TEST :  1.2921745777130127
output_layer_dense_32_128_loss  :  1.2170664072036743 , TEST :  1.279090166091919
output_layer_dense_64_32_loss  :  1.1581077575683594 , TEST :  1.2090129852294922
output_layer_dense_64_64_loss  :  1.3285554647445679 , TEST :  1.3847533464431763
output_layer_dense_64_128_loss  :  1.0729682445526123 , TEST :  1.230327844619751
output_layer_dense_128_32_loss  :  1.249609351158142 , TEST :  1.2621759176254272
output_layer_dense_128_64_loss  :  1.0354626178741455 , TEST :  1.2002227306365967
output_layer_dense_128_128_loss  :  1.279158353805542 , TEST :  1.291643738746643

 
 

loss  :  11.597386360168457 , TEST :  13.247419357299805
output_layer_dense_32_32_32_loss  :  1.0324798822402954 , TEST :  1.1432483196258545
output_layer_dense_32_32_64_loss  :  1.007598042488

In [14]:
opt = model_opt.Model_Optimization(X_train, y_train, X_test, y_test, 200, m.evaluate_dict_list, save_dir = "/home/anish/ASC_ML_test_weights/candidate_models/")

In [15]:
opt.optimize_models(save=True)

--------------------------------------------------------------------------------
Model Name : dense_32_32_64

----------------------------------------------------------------------------------------------------------------
BEST HYPERPARAMETERS : BEST_LOSS : 0.31922340393066406, BEST_ACTIVATION : selu, BEST_INITIALIZER : LecunUniform, BEST_LEARINING_RATE : 0.002, BEST_BATCHSIZE : 16

 
 

loss  :  0.38567402958869934 , TEST :  0.513252317905426


2022-08-31 20:41:19.506763: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /home/anish/ASC_ML_test_weights/candidate_models/dense_32_32_64/assets
--------------------------------------------------------------------------------
Model Name : dense_64_128_32

----------------------------------------------------------------------------------------------------------------
BEST HYPERPARAMETERS : BEST_LOSS : 0.793637216091156, BEST_ACTIVATION : relu, BEST_INITIALIZER : GlorotNormal, BEST_LEARINING_RATE : 0.001, BEST_BATCHSIZE : 16

 
 

loss  :  0.4653832018375397 , TEST :  0.577656090259552
INFO:tensorflow:Assets written to: /home/anish/ASC_ML_test_weights/candidate_models/dense_64_128_32/assets
--------------------------------------------------------------------------------
Model Name : dense_64_64_128

----------------------------------------------------------------------------------------------------------------
BEST HYPERPARAMETERS : BEST_LOSS : 0.4756823480129242, BEST_ACTIVATION : selu, BEST_INITIALIZER : LecunUniform, BEST_

In [18]:
opt.model_confs

[['', 3, 3, 'relu', {'layer1': 32, 'layer2': 32, 'layer3': 64}, [1, None]],
 ['', 3, 3, 'relu', {'layer1': 64, 'layer2': 128, 'layer3': 32}, [1, None]],
 ['', 3, 3, 'relu', {'layer1': 64, 'layer2': 64, 'layer3': 128}, [1, None]],
 ['', 3, 3, 'relu', {'layer1': 32, 'layer2': 32, 'layer3': 128}, [1, None]],
 ['', 3, 3, 'relu', {'layer1': 32, 'layer2': 64, 'layer3': 32}, [1, None]],
 ['', 3, 3, 'relu', {'layer1': 64, 'layer2': 128, 'layer3': 64}, [1, None]],
 ['', 3, 3, 'relu', {'layer1': 32, 'layer2': 128, 'layer3': 64}, [1, None]],
 ['', 3, 3, 'relu', {'layer1': 128, 'layer2': 128, 'layer3': 128}, [1, None]],
 ['', 3, 3, 'relu', {'layer1': 32, 'layer2': 32, 'layer3': 32}, [1, None]],
 ['', 3, 3, 'relu', {'layer1': 128, 'layer2': 32, 'layer3': 32}, [1, None]]]

In [20]:
opt.best_hyp_permodel

[[0.002, 16, 'selu', 'LecunUniform'],
 [0.001, 16, 'relu', 'GlorotNormal'],
 [0.001, 32, 'selu', 'LecunUniform'],
 [0.01, 64, 'selu', 'LecunUniform'],
 [0.001, 32, 'selu', 'LecunUniform'],
 [0.0002, 16, 'selu', 'LecunUniform'],
 [0.0004, 16, 'selu', 'LecunUniform'],
 [0.0005, 16, 'selu', 'LecunNormal'],
 [0.00251, 32, 'selu', 'LecunUniform'],
 [0.00063, 16, 'selu', 'LecunUniform']]

In [18]:
from tensorflow.keras.optimizers import Adam

class Dropout_Optimization():
    def __init__(self, train_x, train_y, test_x, test_y, epochs, model):
        self._train_x = train_x
        self._train_y = train_y
        self._test_x = test_x
        self._test_y = test_y
        self._epochs = epochs
        self._model = model
    
    def get_loss_function(self):
        # Logic to get loss funtion
        # return "mean_absolute_percentage_error"
        # return self.root_mean_squared_error
        # return "mean_squared_error"
        return "mean_absolute_error"

    def dropout_optimization(self, lr = 1e-3, batch_size = 64, epoch = 100):
        loss_fn = self.get_loss_function()
        dropout_indices = self._get_dropout_index()
        dropout_list = [0,0.2,0.5]
        dropout_comb_list = []
        dropout_loss_list = []
        for dropout0 in dropout_list:
            for dropout1 in dropout_list:
                dropout_comb_list.append([dropout0,dropout1])
                self._model.layers[dropout_indices[0]].rate = dropout0
                self._model.layers[dropout_indices[1]].rate = dropout1
                optimizer = Adam(lr = lr)
                self._model.compile(loss = loss_fn, optimizer = optimizer)
                history = self._model.fit(self._train_x, self._train_y, epochs = epoch, batch_size = batch_size, verbose = 0)

                scores = self._model.evaluate(self._train_x, self._train_y, verbose = 0)
                scores_test = self._model.evaluate(self._test_x, self._test_y, verbose = 0)
#                 print(f"For Dropouts D0 = {dropout0}, D1 = {dropout1}, TRAIN_LOSS = {scores}, TEST_LOSS = {scores_test}")
                dropout_loss_list.append(scores_test)
                
        best_dropout_rates = dropout_comb_list[dropout_loss_list.index(min(dropout_loss_list))]
        self._model.layers[dropout_indices[0]].rate = best_dropout_rates[0]
        self._model.layers[dropout_indices[1]].rate = best_dropout_rates[1]
        return best_dropout_rates
                

    def _get_dropout_index(self):
        dropout_indices = []
        ind = 0
        for layer in self._model.layers:
            if layer.__class__.__name__ == "Dropout":
                dropout_indices.append(ind)
            ind = ind+1
        return dropout_indices

In [23]:
import os
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.initializers import RandomUniform, GlorotUniform, GlorotNormal, HeUniform, HeNormal

from ASC_ML.networkbuilding import model_generation as model_gen
from ASC_ML.networkbuilding import hyperparameter_optimization as hyp_opt

class Model_Stacking2:
    def __init__(self, train_x, train_y, test_x, test_y, model_path_list, model_conf_list, save_dir = ""):
        self._model_path_list = model_path_list
        self._model_conf_list = model_conf_list
        self._train_x = train_x
        self._train_y = train_y
        self._test_x = test_x
        self._test_y = test_y
        self._save_dir = save_dir
        self._stacked_model_paths = []
    
    def get_loss_function(self):
        # Logic to get loss funtion
        # return "mean_absolute_percentage_error"
        # return self.root_mean_squared_error
        # return "mean_squared_error"
        return "mean_absolute_error"

    def _stacked_model_generator(self):
        for path in self._model_path_list:
            for model_conf in self._model_conf_list:
                model1 = load_model(path)
                activation = model1.layers[-2].get_config()["activation"]
                reduced_model1 = Model(name = model1.name+"_reduced", inputs = model1.input, outputs = model1.layers[-2].output)
                last_layer = reduced_model1.output
                x = last_layer
                model2_obj = model_gen.NN_ModelGeneration(*model_conf)
                x = Dropout(0.0)(x)
                for layer_name in model2_obj.layer_conf:
                    x = Dense(model2_obj.layer_conf[layer_name], activation = activation, name = layer_name+"_2nd")(x)
                x = Dropout(0.0)(x)
                output_layer = Dense(model2_obj.output_layer_conf[0], activation = model2_obj.output_layer_conf[1], name = "output_layer" + "_" + model2_obj.model_name)(x)

                stacked_model = Model(name = model1.name + "_st_" + model2_obj.model_name,inputs = reduced_model1.input, outputs = output_layer)
                x = None
                output_layer = None
                yield stacked_model

    def optimize_stacked_models(self):
        loss_fn = self.get_loss_function()
        stacked_model_generator = self._stacked_model_generator()
        for model in stacked_model_generator:
            print(model.name)
            print(model.summary())
            h = hyp_opt.Hyperparameter_Optimization([self._train_x], [self._train_y], model, loss_fn, activation_opt = True, initializer_opt = True)
            best_lr, best_batch_size, _, best_initializer = h.get_best_hyperparameters()
            print(best_initializer)
            self._reinitialize_model(model, best_initializer)
            
            dr = Dropout_Optimization(self._train_x, self._train_y, self._test_x, self._test_y, epochs = 100, model = model)
            best_dropout_rates = dr.dropout_optimization(lr = best_lr, batch_size = best_batch_size, epoch = 100)
            print(f"DROPOUT RATES : {best_dropout_rates}")
            
            model = self._train_models(model = model, lr = best_lr, batch_size = best_batch_size)
            self._save_model(model)
            
#             print(f"BEST LR STACKED {best_lr}, BEST BATCH SIZE {best_batch_size}")
#             dr = Dropout_Optimization(self._train_x, self._train_y, self._test_x, self._test_y, epochs = 100, model = model)
#             dr.dropout_optimization(lr = best_lr, batch_size = best_batch_size, epoch = 100)
    
    def _train_models(self, model, lr, batch_size):
        loss_fn = self.get_loss_function()
        optimizer = Adam(lr = lr)
        model.compile(loss = loss_fn, optimizer = optimizer)
        history = model.fit(self._train_x, self._train_y, epochs = 100, batch_size = batch_size, verbose = 0)

        scores = model.evaluate(self._train_x, self._train_y, verbose = 0)
        scores_test = model.evaluate(self._test_x, self._test_y, verbose = 0)
        print(f"TRAIN_LOSS = {scores}, TEST_LOSS = {scores_test}")
        return model
        
    @staticmethod
    def _reinitialize_model(model, initializer_str = "RandomUniform"):
        if initializer_str == "RandomUniform":
            initializer = RandomUniform(seed = 420)
        elif initializer_str == "GlorotUniform":
            initializer = GlorotUniform(seed = 420)
        elif initializer_str == "HeUniform":
            initializer = HeUniform(seed = 420)
        elif initializer_str == "GlorotNormal":
            initializer = GlorotNormal(seed = 420)
        elif initializer_str == "HeNormal":
            initializer = HeNormal(seed = 420)
        for layer in model.layers:
            layer.set_weights([initializer(shape=w.shape) for w in layer.get_weights()])
            
    def _save_model(self, model):
        path = os.path.join(self._save_dir,model.name)
        model.save(path)
        self._stacked_model_paths.append(path)


In [22]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [23]:
stacked_models = model_stacking.Model_Stacking(X_train, y_train, X_test, y_test, opt.saved_paths, opt.model_confs, save_dir = "/home/anish/ASC_ML_test_weights/stacked_models/")
stacked_models.optimize_stacked_models()

dense_32_32_64_st_dense_32_32_64
Model: "dense_32_32_64_st_dense_32_32_64"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer_dense_32_32_64 ( [(None, 3)]               0         
_________________________________________________________________
layer1_dense_32_32_64 (Dense (None, 32)                128       
_________________________________________________________________
layer2_dense_32_32_64 (Dense (None, 32)                1056      
_________________________________________________________________
layer3_dense_32_32_64 (Dense (None, 64)                2112      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
layer1_dense_32_32_64_2nd (D (None, 32)                2080      
_________________________________________________________________
l

KeyboardInterrupt: 

In [25]:
# stacked_models = model_stacking.Model_Stacking(opt.saved_paths, opt.model_confs)
stacked_models = Model_Stacking2(X_train, y_train, X_test, y_test, opt.saved_paths, opt.model_confs, save_dir = "/home/anish/ASC_ML_test_weights/stacked_models/")

In [26]:
stacked_models.optimize_stacked_models()

dense_64_128_32_st_dense_64_128_32
Model: "dense_64_128_32_st_dense_64_128_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer_dense_64_128_32  [(None, 3)]               0         
_________________________________________________________________
layer1_dense_64_128_32 (Dens (None, 64)                256       
_________________________________________________________________
layer2_dense_64_128_32 (Dens (None, 128)               8320      
_________________________________________________________________
layer3_dense_64_128_32 (Dens (None, 32)                4128      
_________________________________________________________________
dropout_6 (Dropout)          (None, 32)                0         
_________________________________________________________________
layer1_dense_64_128_32_2nd ( (None, 64)                2112      
_______________________________________________________________

DROPOUT RATES : [0, 0]
TRAIN_LOSS = 0.20413604378700256, TEST_LOSS = 0.3925964832305908
dense_64_128_32_st_dense_32_128
Model: "dense_64_128_32_st_dense_32_128"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer_dense_64_128_32  [(None, 3)]               0         
_________________________________________________________________
layer1_dense_64_128_32 (Dens (None, 64)                256       
_________________________________________________________________
layer2_dense_64_128_32 (Dens (None, 128)               8320      
_________________________________________________________________
layer3_dense_64_128_32 (Dens (None, 32)                4128      
_________________________________________________________________
dropout_14 (Dropout)         (None, 32)                0         
_________________________________________________________________
layer1_dense_32_128_2nd (Den (None, 32)        

DROPOUT RATES : [0, 0]
TRAIN_LOSS = 0.3202672004699707, TEST_LOSS = 0.47966986894607544
dense_64_128_32_st_dense_128_128_32
Model: "dense_64_128_32_st_dense_128_128_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer_dense_64_128_32  [(None, 3)]               0         
_________________________________________________________________
layer1_dense_64_128_32 (Dens (None, 64)                256       
_________________________________________________________________
layer2_dense_64_128_32 (Dens (None, 128)               8320      
_________________________________________________________________
layer3_dense_64_128_32 (Dens (None, 32)                4128      
_________________________________________________________________
dropout_22 (Dropout)         (None, 32)                0         
_________________________________________________________________
layer1_dense_128_128_32_2nd  (None, 128

KeyboardInterrupt: 

In [58]:
from tensorflow.keras.models import load_model
model = load_model("/home/anish/ASC_ML_test_weights/candidate_models/dense_64_64_64/")

In [59]:
model.summary()

Model: "dense_64_64_64"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer_dense_64_64_64 ( [(None, 3)]               0         
_________________________________________________________________
layer1_dense_64_64_64 (Dense (None, 64)                256       
_________________________________________________________________
layer2_dense_64_64_64 (Dense (None, 64)                4160      
_________________________________________________________________
layer3_dense_64_64_64 (Dense (None, 64)                4160      
_________________________________________________________________
output_layer_dense_64_64_64  (None, 1)                 65        
Total params: 8,641
Trainable params: 8,641
Non-trainable params: 0
_________________________________________________________________


In [60]:
activation = model.layers[-2].get_config()["activation"]
print(activation)
print(type(activation))

selu
<class 'str'>


In [61]:
model.summary()

Model: "dense_64_64_64"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer_dense_64_64_64 ( [(None, 3)]               0         
_________________________________________________________________
layer1_dense_64_64_64 (Dense (None, 64)                256       
_________________________________________________________________
layer2_dense_64_64_64 (Dense (None, 64)                4160      
_________________________________________________________________
layer3_dense_64_64_64 (Dense (None, 64)                4160      
_________________________________________________________________
output_layer_dense_64_64_64  (None, 1)                 65        
Total params: 8,641
Trainable params: 8,641
Non-trainable params: 0
_________________________________________________________________


In [62]:
reduce_model = Model(name = model.name+"_reduced", inputs = model.input, outputs = model.layers[-2].output)
print(reduce_model.summary())

Model: "dense_64_64_64_reduced"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer_dense_64_64_64 ( [(None, 3)]               0         
_________________________________________________________________
layer1_dense_64_64_64 (Dense (None, 64)                256       
_________________________________________________________________
layer2_dense_64_64_64 (Dense (None, 64)                4160      
_________________________________________________________________
layer3_dense_64_64_64 (Dense (None, 64)                4160      
Total params: 8,576
Trainable params: 8,576
Non-trainable params: 0
_________________________________________________________________
None


In [63]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Concatenate, Flatten, Input, Dropout

In [64]:
# new_model = Sequential()
# new_model.add(Dropout(0.1))
# new_model.add(reduce_model)
# new_model.add(Dropout(0.2))
# new_model.add(Dense(1024, activation = activation))
# new_model.add(Dense(1024, activation = activation))
# new_model.add(Dense(1))

In [65]:
last_layer = reduce_model.output
x = last_layer
x = Dropout(0.1)(x)
x = Dense(1024, activation = activation)(x)
x = Dense(1024, activation = activation)(x)
x = Dropout(0.1)(x)
x = Dense(1)(x)
new_model = Model(inputs = reduce_model.input, outputs = x)

In [66]:
new_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer_dense_64_64_64 ( [(None, 3)]               0         
_________________________________________________________________
layer1_dense_64_64_64 (Dense (None, 64)                256       
_________________________________________________________________
layer2_dense_64_64_64 (Dense (None, 64)                4160      
_________________________________________________________________
layer3_dense_64_64_64 (Dense (None, 64)                4160      
_________________________________________________________________
dropout_326 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1024)              66560     
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              104960

In [69]:
dropout_ind = []
ind = 0
for layer in new_model.layers:
    if layer.__class__.__name__ == "Dropout":
        dropout_ind.append(ind)
    ind = ind+1
print(dropout_ind)

[4, 7]


In [71]:
new_model.layers[4].rate = 0.5

In [72]:
new_model.layers[4].rate

0.5

In [20]:
from sklearn.preprocessing import LabelEncoder

# load the sonar dataset
from pandas import read_csv
# load dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/auto-insurance.csv'
dataframe = read_csv(url, header=None)
print(dataframe.shape)
# split into input and output elements
data = dataframe.values
data = data.astype('float32')
X, y = data[:, :-1], data[:, -1]
print(X.shape, y.shape)
# separate into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(63, 2)
(63, 1) (63,)
(42, 1) (21, 1) (42,) (21,)


In [22]:
start = time.time()
m = multiple.Multiple_Model_Gen_V3(X_train, y_train, X_test, y_test, 50, 64, input_shape = 1, 
                                   max_no_layers = 3, model_per_batch = 10, 
                                   save_dir = "/home/anish/ASC_ML_test_weights/")
m.get_model_confs()
m.get_best_models(save = True)
end = time.time()
print("Time Taken : ", (end-start))


 
 

loss  :  564.4772338867188 , TEST :  696.4302978515625
output_layer_dense_32_32_loss  :  77.97647094726562 , TEST :  102.96764373779297
output_layer_dense_32_64_loss  :  72.4428939819336 , TEST :  94.78680419921875
output_layer_dense_32_128_loss  :  43.49039840698242 , TEST :  50.427490234375
output_layer_dense_32_1024_loss  :  32.640380859375 , TEST :  31.093881607055664
output_layer_dense_64_32_loss  :  68.5215072631836 , TEST :  88.98941802978516
output_layer_dense_64_64_loss  :  67.38311004638672 , TEST :  87.2926254272461
output_layer_dense_64_128_loss  :  41.06880569458008 , TEST :  45.563011169433594
output_layer_dense_64_1024_loss  :  32.03666687011719 , TEST :  28.78836441040039
output_layer_dense_128_32_loss  :  72.19493103027344 , TEST :  94.40507507324219
output_layer_dense_128_64_loss  :  56.72202682495117 , TEST :  72.11593627929688

 
 

loss  :  204.6041717529297 , TEST :  193.50648498535156
output_layer_dense_128_128_loss  :  43.74370574951172 , TEST :  50.915996

INFO:tensorflow:Assets written to: /home/anish/ASC_ML_test_weights/dense_128_1024_32/assets
INFO:tensorflow:Assets written to: /home/anish/ASC_ML_test_weights/dense_32_1024_1024/assets
INFO:tensorflow:Assets written to: /home/anish/ASC_ML_test_weights/dense_1024_128/assets
INFO:tensorflow:Assets written to: /home/anish/ASC_ML_test_weights/dense_1024_1024_1024/assets
INFO:tensorflow:Assets written to: /home/anish/ASC_ML_test_weights/dense_64_128_32/assets
INFO:tensorflow:Assets written to: /home/anish/ASC_ML_test_weights/dense_32_32_128/assets
INFO:tensorflow:Assets written to: /home/anish/ASC_ML_test_weights/dense_64_64_1024/assets
INFO:tensorflow:Assets written to: /home/anish/ASC_ML_test_weights/dense_1024_1024_64/assets
INFO:tensorflow:Assets written to: /home/anish/ASC_ML_test_weights/dense_32_1024_64/assets
INFO:tensorflow:Assets written to: /home/anish/ASC_ML_test_weights/dense_32_1024_128/assets

-------------------------------------------------------------------------------
Tim

In [23]:
opt = model_opt.Model_Optimization(X_train, y_train, X_test, y_test, 200, m.evaluate_dict_list)
opt.optimize_models()

Model Name : dense_128_1024_32

----------------------------------------------------------------------------------------------------------------
BEST HYPERPARAMETERS : BEST_LOSS : 29.345319747924805, BEST_ACTIVATION : selu, BEST_INITIALIZER : GlorotNormal, BEST_LEARINING_RATE : 0.02512, BEST_BATCHSIZE : 16

 
 

loss  :  90.03191375732422 , TEST :  119.77239227294922
Model Name : dense_32_1024_1024

----------------------------------------------------------------------------------------------------------------
BEST HYPERPARAMETERS : BEST_LOSS : 29.84622573852539, BEST_ACTIVATION : selu, BEST_INITIALIZER : GlorotNormal, BEST_LEARINING_RATE : 0.00158, BEST_BATCHSIZE : 16

 
 

loss  :  28.873470306396484 , TEST :  24.018808364868164
Model Name : dense_1024_128

----------------------------------------------------------------------------------------------------------------
BEST HYPERPARAMETERS : BEST_LOSS : 29.384897232055664, BEST_ACTIVATION : selu, BEST_INITIALIZER : GlorotNormal, BEST_


 
 

loss  :  90.03191375732422 , TEST :  119.77239227294922
Model Name : dense_32_1024_64

----------------------------------------------------------------------------------------------------------------
BEST HYPERPARAMETERS : BEST_LOSS : 29.205533981323242, BEST_ACTIVATION : selu, BEST_INITIALIZER : GlorotUniform, BEST_LEARINING_RATE : 0.01259, BEST_BATCHSIZE : 16

 
 

loss  :  29.2604923248291 , TEST :  26.66190528869629
Model Name : dense_32_1024_128

----------------------------------------------------------------------------------------------------------------
BEST HYPERPARAMETERS : BEST_LOSS : 29.960067749023438, BEST_ACTIVATION : selu, BEST_INITIALIZER : GlorotUniform, BEST_LEARINING_RATE : 0.002, BEST_BATCHSIZE : 16

 
 

loss  :  28.737110137939453 , TEST :  25.75638198852539


In [ ]:
search = StructuredDataRegressor(max_trials=15, loss='mean_absolute_error')

In [ ]:
search.fit(x=X_train, y=y_train)

In [ ]:
train_mae, _ = search.evaluate(X_train, y_train, verbose=0)
print('Mean Absolute Error is {}'.format(train_mae))

In [ ]:
test_mae, _ = search.evaluate(X_test, y_test, verbose=0)
print('Mean Absolute Error is {}'.format(test_mae))